<div class="alert alert-block alert-success">
<font color="blue" size="4em">Source</font><br>
http://www.cine21.com/db/person
</div>

- Request URL: http://www.cine21.com/rank/person/content
- Request Method: POST
- Form Data
- section:actor
- period_start:2021-04
- gender:all
- page:1

In [12]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

### MongoDB connection

In [3]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

### 크롤링 주소 requests

In [7]:
cine21_url = "http://www.cine21.com/rank/person/content"
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-10'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(cine21_url, data=post_data)

### parsing과 데이터 추출

In [8]:
soup = BeautifulSoup(res.content, 'html.parser')

In [17]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(re.sub('\(\w+\)', '', actor.text))

조우진
김윤석
조인성
성동일
이제훈
허준호
김소진


In [71]:
actor_info_list = list()

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')


for idx, actor in enumerate(actors):    
    
    
    actor_name = re.sub('\(\w+\)', '', actor.text)
    actor_hits = int(hits[idx].text.replace(',',''))
    movie_titles = movies[idx].select('li a span')
    movie_list = []
    for movie_title in movie_titles:
        movie_list.append(movie_title.text)
    
    actor_link = 'http://www.cine21.com/' + actor.select_one('a')['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    actor_info_dict = dict()
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연영화'] = movie_list
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value
        
    actor_info_list.append(actor_info_dict)

actor_info_list

[{'배우이름': '조우진',
  '흥행지수': 41982,
  '출연영화': ['발신제한', '도굴', '서복', '국가부도의 날', '봉오동 전투', '강철비'],
  '다른 이름': '조신제',
  '직업': '배우',
  '생년월일': '1979-01-16',
  '성별': '남'},
 {'배우이름': '김윤석',
  '흥행지수': 37394,
  '출연영화': ['모가디슈', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '1987', '도둑들'],
  '직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'배우이름': '조인성',
  '흥행지수': 32750,
  '출연영화': ['모가디슈', '클래식'],
  '다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'배우이름': '성동일',
  '흥행지수': 30742,
  '출연영화': ['담보', '수상한 그녀'],
  '직업': '배우',
  '생년월일': '1967-04-27',
  '성별': '남',
  '신장/체중': '177cm, 78kg',
  '학교': '유한대학 기계설계학'},
 {'배우이름': '이제훈',
  '흥행지수': 30681,
  '출연영화': ['도굴', '사냥의 시간', '건축학개론', '아이 캔 스피크', '파수꾼', '박열'],
  '직업': '배우',
  '생년월일': '1984-07-04',
  '성별': '남',
  '홈페이지': '\nhttps

### 배우 정보 추출 및 몽고DB 저장

In [82]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection



actor_info_list = list()

cine21_url = "http://www.cine21.com/rank/person/content"
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-10'
post_data['gender'] = 'all'

for i in range(1, 21):
    post_data['page'] = i

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')



    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')

    for idx, actor in enumerate(actors):    
        actor_name = re.sub('\(\w+\)', '', actor.text)
        actor_hits = int(hits[idx].text.replace(',',''))
        movie_titles = movies[idx].select('li a span')
        movie_list = []
        for movie_title in movie_titles:
            movie_list.append(movie_title.text)

        actor_link = 'http://www.cine21.com/' + actor.select_one('a')['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_list
        actor_info_dict['랭킹'] = rankings[idx].text

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value

        actor_info_list.append(actor_info_dict)

actor_collection.insert_many(actor_info_list)